In [1]:
import pytorch_lightning as pl

/opt/homebrew/lib/python3.9/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

from itertools import islice

In [3]:
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

In [4]:
from pts.model.dnri import DNRIEstimator
from pts.modules import StudentTOutput

In [5]:
dataset = get_dataset("exchange_rate_nips", regenerate=False)

In [6]:
dataset.metadata

MetaData(freq='B', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat_0', cardinality='8')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=30)

In [7]:
train_grouper = MultivariateGrouper(max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(dataset.train)), 
                                   max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

In [8]:
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

/opt/homebrew/lib/python3.9/site-packages/gluonts/dataset/common.py:324: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(timestamp_input, freq=freq)
/opt/homebrew/lib/python3.9/site-packages/gluonts/dataset/common.py:327: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  if isinstance(timestamp.freq, Tick):
/opt/homebrew/lib/python3.9/site-packages/gluonts/dataset/common.py:338: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  return timestamp.freq.rollforward(timestamp)
/opt/homebrew/lib/python3.9/site-packages/gluonts/dataset/multivariate_grouper.py:106: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  timestamp + (len(data[FieldName.TARGET]) - 1) * timestamp.freq,
/opt/homebrew/lib/python3.9/site-packages/gluonts/dataset/multivariate_grouper.py:108: FutureWarning: Timestamp.freq is deprecate

In [11]:
estimator = DNRIEstimator(
    freq=dataset.metadata.freq,
    context_length=dataset.metadata.prediction_length,
    prediction_length=dataset.metadata.prediction_length,
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    distr_output=StudentTOutput(int(dataset.metadata.feat_static_cat[0].cardinality)),
    mlp_hidden_size=16,
    decoder_hidden=16,
    rnn_hidden_size=16,
    
    batch_size=64,
    num_batches_per_epoch=100,
    trainer_kwargs=dict(max_epochs=10),
)   

In [ ]:
predictor = estimator.train(
    training_data=dataset_train,
    num_workers=4,
    shuffle_buffer_length=1024
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type      | Params
------------------------------------
0 | model | DNRIModel | 15.9 K
------------------------------------
15.9 K    Trainable params
0         Non-trainable params
15.9 K    Total params
0.064     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]